# Distributed Tensorflow with Horovod
In this tutorial, you will train a word2vec model in TensorFlow using distributed training via [Horovod](https://github.com/uber/horovod).

> https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-tensorflow

<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>

The TensorFlow estimator also supports distributed training across CPU and GPU clusters. You can easily run distributed TensorFlow jobs and Azure Machine Learning will manage the orchestration for you.

Azure Machine Learning supports two methods of distributed training in TensorFlow:

- **MPI-based distributed training** using the Horovod framework<br>
- **Native distributed TensorFlow** using the parameter server method

**Horovod**
Horovod is an open-source framework for distributed training developed by Uber. It offers an easy path to distributed GPU TensorFlow jobs.

To use Horovod, specify an **MpiConfiguration object** for the distributed_training parameter in the TensorFlow constructor. This parameter ensures that Horovod library is installed for you to use in your training script.

## 1. Infos

In [31]:
import sys
sys.version

'3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) \n[GCC 7.3.0]'

In [32]:
import datetime
maintenant = datetime.datetime.now()
print('Date :', maintenant)

Date : 2020-03-11 08:54:35.669647


In [33]:
import azureml.core
print("Azure ML version :", azureml.core.VERSION)

Azure ML version : 1.0.83


In [34]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: AzureMLWorkshop
Azure region: westeurope
Resource group: AzureMLWorkshopRG


## 2. Création compute GPU

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "gpuclusterNC6"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

print(compute_target.status.serialize())

Creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-03-11T08:29:36.781000+00:00', 'errors': None, 'creationTime': '2020-03-11T08:29:23.360394+00:00', 'modifiedTime': '2020-03-11T08:29:46.789092+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


## 3. Chargement des données

In [4]:
import os
import urllib

os.makedirs('./data', exist_ok=True)
download_url = 'http://mattmahoney.net/dc/text8.zip'
urllib.request.urlretrieve(download_url, filename='./data/text8.zip')

('./data/text8.zip', <http.client.HTTPMessage at 0x7f3fa7a26ef0>)

Each workspace is associated with a default datastore. In this tutorial, we will upload the training data to this default datastore.

In [5]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

AzureBlob azuremlworksho6034843387 azureml-blobstore-3ead7922-76f5-437e-9c14-08f627fe3f3f


Upload the contents of the data directory to the path `./data` on the default datastore.

In [6]:
ds.upload(src_dir='data', target_path='data', overwrite=True, show_progress=True)

Uploading an estimated of 5 files
Uploading data/mnist/test-images.gz
Uploading data/mnist/test-labels.gz
Uploading data/mnist/train-images.gz
Uploading data/mnist/train-labels.gz
Uploading data/text8.zip
Uploaded data/mnist/train-labels.gz, 1 files out of an estimated total of 5
Uploaded data/mnist/test-labels.gz, 2 files out of an estimated total of 5
Uploaded data/mnist/test-images.gz, 3 files out of an estimated total of 5
Uploaded data/mnist/train-images.gz, 4 files out of an estimated total of 5
Uploaded data/text8.zip, 5 files out of an estimated total of 5
Uploaded 5 files


$AZUREML_DATAREFERENCE_485b538164b242c9a460e58b42d8b40e

For convenience, let's get a reference to the path on the datastore with the zip file of training data. We can do so using the `path` method. In the next section, we can then pass this reference to our training script's `--input_data` argument. 

In [7]:
path_on_datastore = 'data/text8.zip'
ds_data = ds.path(path_on_datastore)
print(ds_data)

$AZUREML_DATAREFERENCE_5a4a00570ad0485b8ff8e80b3e1991b5


## 4. Modèle

In [8]:
import os

project_folder = './tf-distr-hvd'
os.makedirs(project_folder, exist_ok=True)

Copy the training script `tf_horovod_word2vec.py` into this project directory.

In [9]:
import shutil

shutil.copy('tf_horovod_word2vec.py', project_folder)

'./tf-distr-hvd/tf_horovod_word2vec.py'

In [10]:
from azureml.core import Experiment

experiment_name = 'Exemple13-TFHorovod'
experiment = Experiment(ws, name=experiment_name)

### 4.1 Estimateur TensorFlow
The AML SDK's TensorFlow estimator enables you to easily submit TensorFlow training jobs for both single-node and distributed runs. For more information on the TensorFlow estimator, refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-train-tensorflow).

In [11]:
from azureml.train.dnn import TensorFlow

script_params={
    '--input_data': ds_data
}

estimator= TensorFlow(source_directory=project_folder,
                      compute_target=compute_target,
                      script_params=script_params,
                      entry_script='tf_horovod_word2vec.py',
                      node_count=2,
                      process_count_per_node=1,
                      distributed_backend='mpi', #Pour horovod
                      use_gpu=True)

WARNING - 'distributed_backend' parameter will be deprecated. Please use 'distributed_training' instead.
WARNING - framework_version is not specified, defaulting to version 1.13.


The above code specifies that we will run our training script on `2` nodes, with one worker per node. In order to execute a distributed run using MPI/Horovod, you must provide the argument `distributed_backend='mpi'`. Using this estimator with these settings, TensorFlow, Horovod and their dependencies will be installed for you. However, if your script also uses other packages, make sure to install them via the `TensorFlow` constructor's `pip_packages` or `conda_packages` parameters.

Note that we passed our training data reference `ds_data` to our script's `--input_data` argument. This will 1) mount our datastore on the remote compute and 2) provide the path to the data zip file on our datastore.

### 4.2 Run

> Nécessite 10 minutes de temps de traitement

In [12]:
run = experiment.submit(estimator)
print(run)

Run(Experiment: Exemple13-TFHorovod,
Id: Exemple13-TFHorovod_1583915733_d7ca0d72,
Type: azureml.scriptrun,
Status: Starting)


### 4.3 Suivi du run

In [13]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [25]:
# Statut du run
run.get_details()

{'runId': 'Exemple13-TFHorovod_1583915733_d7ca0d72',
 'target': 'gpuclusterNC6',
 'status': 'Completed',
 'startTimeUtc': '2020-03-11T08:49:40.928255Z',
 'endTimeUtc': '2020-03-11T08:52:58.695681Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '51eedd39-7731-480d-bc6b-f845adb6aa52',
  'azureml.git.repository_uri': 'https://github.com/retkowsky/WorkshopAML2020',
  'mlflow.source.git.repoURL': 'https://github.com/retkowsky/WorkshopAML2020',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': '92bcd73fc9ec1037078710902a207fd495a95825',
  'mlflow.source.git.commit': '92bcd73fc9ec1037078710902a207fd495a95825',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'runDefinition': {'script': 'tf_horovod_word2vec.py',
  'useAbsolutePath': False,
  'arguments': ['--input_data',
   '$AZUREML_DATAREF

In [26]:
run.get_metrics()

{'Loss': [267.4751892089844,
  272.8165588378906,
  87.33021711301804,
  86.38771854686738]}

In [27]:
compute_target.get_status().serialize()

{'currentNodeCount': 2,
 'targetNodeCount': 2,
 'nodeStateCounts': {'preparingNodeCount': 0,
  'runningNodeCount': 2,
  'idleNodeCount': 0,
  'unusableNodeCount': 0,
  'leavingNodeCount': 0,
  'preemptedNodeCount': 0},
 'allocationState': 'Steady',
 'allocationStateTransitionTime': '2020-03-11T08:49:18.083000+00:00',
 'errors': None,
 'creationTime': '2020-03-11T08:29:23.360394+00:00',
 'modifiedTime': '2020-03-11T08:29:46.789092+00:00',
 'provisioningState': 'Succeeded',
 'provisioningStateTransitionTime': None,
 'scaleSettings': {'minNodeCount': 0,
  'maxNodeCount': 4,
  'nodeIdleTimeBeforeScaleDown': 'PT120S'},
 'vmPriority': 'Dedicated',
 'vmSize': 'STANDARD_NC6'}

In [28]:
# Statut
compute_target.list_nodes()

[{'nodeId': 'tvmps_30394c93ebc0fd30b3411f1e1699269c091cb625737a22d01691584cfc91d01d_d',
  'port': 50001,
  'publicIpAddress': '40.114.180.54',
  'privateIpAddress': '10.0.0.5',
  'nodeState': 'idle'},
 {'nodeId': 'tvmps_7ea46a5fa897ca7d33312d8c01a9c81d42f92e52781df08bcf651af07a7f80d3_d',
  'port': 50000,
  'publicIpAddress': '40.114.180.54',
  'privateIpAddress': '10.0.0.4',
  'nodeState': 'idle'}]

In [29]:
run.wait_for_completion(show_output=True)

RunId: Exemple13-TFHorovod_1583915733_d7ca0d72
Web View: https://ml.azure.com/experiments/Exemple13-TFHorovod/runs/Exemple13-TFHorovod_1583915733_d7ca0d72?wsid=/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/AzureMLWorkshopRG/workspaces/AzureMLWorkshop

Execution Summary
RunId: Exemple13-TFHorovod_1583915733_d7ca0d72
Web View: https://ml.azure.com/experiments/Exemple13-TFHorovod/runs/Exemple13-TFHorovod_1583915733_d7ca0d72?wsid=/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/AzureMLWorkshopRG/workspaces/AzureMLWorkshop



{'runId': 'Exemple13-TFHorovod_1583915733_d7ca0d72',
 'target': 'gpuclusterNC6',
 'status': 'Completed',
 'startTimeUtc': '2020-03-11T08:49:40.928255Z',
 'endTimeUtc': '2020-03-11T08:52:58.695681Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '51eedd39-7731-480d-bc6b-f845adb6aa52',
  'azureml.git.repository_uri': 'https://github.com/retkowsky/WorkshopAML2020',
  'mlflow.source.git.repoURL': 'https://github.com/retkowsky/WorkshopAML2020',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': '92bcd73fc9ec1037078710902a207fd495a95825',
  'mlflow.source.git.commit': '92bcd73fc9ec1037078710902a207fd495a95825',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'runDefinition': {'script': 'tf_horovod_word2vec.py',
  'useAbsolutePath': False,
  'arguments': ['--input_data',
   '$AZUREML_DATAREF

In [30]:
experiment

Name,Workspace,Report Page,Docs Page
Exemple13-TFHorovod,AzureMLWorkshop,Link to Azure Machine Learning studio,Link to Documentation


<img src="https://github.com/retkowsky/images/blob/master/Powered-by-MS-Azure-logo-v2.png?raw=true" height="300" width="300">